<a href="https://colab.research.google.com/github/bmox/tools/blob/main/download_file_from_internet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title ##**Select your storage type** { display-mode: "form" }

!pip install kora
!pip install youtube_dl
import requests 
from IPython.display import clear_output
from google.colab import drive
from google.colab import files
import glob
import os
import math
from kora import drive
import subprocess
clear_output()
storage="temporary"#@param["temporary","google drive"]
if storage=="google drive":
  drive.mount('/content/gdrive')
  os.chdir("/content/gdrive/My Drive")
  # clear_output()
  try:
    os.mkdir("download_from_colab")
  except:
    pass
  os.chdir("/content/gdrive/My Drive/download_from_colab")
  print("Using drive as a storage")
elif storage=="temporary":
  os.chdir("/content/")
  try:
    os.mkdir("download_from_colab")
  except:
    pass
  os.chdir("/content/download_from_colab")
  print("Using colab as a temporary storage") 

Using colab as a temporary storage


In [2]:
# @title ##**If you download from link** { display-mode: "form" }

download_link= "https://assets.mixkit.co/videos/preview/mixkit-fireworks-illuminating-the-beach-sky-4157-large.mp4"#@param {type:"string"}
r = requests.get(download_link) 
file_name="movie.mp4"#@param {type:"string"}
with open(file_name,'wb') as f: 
	f.write(r.content) 
print("Download Successful!")

Download Successful!


In [13]:
# @title ##**Download file from drive link** { display-mode: "form" }
# #@markdown ### Default leave it blank (If you not select "Download from link" ):
drive_download_link = " https://drive.google.com/file/d/1J7yc_zC1D11BMJyQpl4Wx1GpSHgJjr-u/view?usp=sharing" #@param {type:"string"}
# Define some helper functions for downloading pretrained model
# taken from this StackOverflow answer: https://stackoverflow.com/a/39225039


def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


       
file_id = drive_download_link.split('/')[-2]
give_file_name ="image.jpg"#@param {type:"string"}
try:
  download_file_from_google_drive(file_id, give_file_name)
  print("Download successful!")
except:
  print("Download failed try again")


Download successful!


In [15]:
# @title ##**Download folder from drive link** { display-mode: "form" }
drive_folder_download_link = "https://drive.google.com/drive/folders/1i2Vju11AG9fgSI4NQIVBJmeCeG-fMNxR?usp=sharing" #@param {type:"string"}
folder_id = drive_folder_download_link.split('/')[-1].split("?")[0]
drive.download_folder(folder_id)
clear_output()
print("Download successful!")

Download successful!


In [4]:
# @title ##**Know the file size** { display-mode: "form" }

def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i]) 
enter_the_file_path="/content/download_from_colab/movie.mp4"#@param {type:"string"}   
size= os.path.getsize(enter_the_file_path)
print(f"File size: {convert_size(size)}")   

File size: 7.45 MB


In [6]:
# @title ##**Google.py** { display-mode: "form" }
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request


def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt


In [9]:
# @title ##**Upload youtube client secret Key and token_youtube_v3.pickel file** { display-mode: "form" }
print("Upload client secret Key")
uploaded = files.upload()
clear_output()
client_secrect_key=glob.glob("./*.json")[0].split("/")[-1]
print(f"{client_secrect_key} upload succcessful!")
print("\n")
print("Upload pickel file")
uploaded = files.upload()
clear_output()
pickle_file=glob.glob("./*.pickle")[0].split("/")[-1]
print(f"{pickle_file} upload succcessful!")

token_youtube_v3.pickle upload succcessful!


#Must Download the image:<br>
copy the link and paste in Download from drive link <br>
drive_download_link:
https://drive.google.com/file/d/1J7yc_zC1D11BMJyQpl4Wx1GpSHgJjr-u/view?usp=sharing<br>
give_file_name: image.jpg



In [10]:
# @title ##**Upload on youtube** { display-mode: "form" }
#https://learndataanalysis.org/how-to-upload-a-video-to-youtube-using-youtube-data-api-in-python/
title_name="video_upload"#@param {type:"string"}
enter_video_file_name="movie.mp4"#@param {type:"string"} 
import datetime
# from Google import Create_Service
from googleapiclient.http import MediaFileUpload

CLIENT_SECRET_FILE = client_secrect_key
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

upload_date_time = datetime.datetime(2025, 12, 25, 12, 30, 0).isoformat() + '.000Z'

request_body = {
    'snippet': {
        'categoryI': 19,
        'title': title_name,
        'description': 'Movie upload',
        'tags': ['video', 'video test', ' upload']
    },
    'status': {
        'privacyStatus': 'private',
        'publishAt': upload_date_time,
        'selfDeclaredMadeForKids': False, 
    },
    'notifySubscribers': False
}

mediaFile = MediaFileUpload(enter_video_file_name)

response_upload = service.videos().insert(
    part='snippet,status',
    body=request_body,
    media_body=mediaFile
).execute()


service.thumbnails().set(
    videoId=response_upload.get('id'),
    media_body=MediaFileUpload('image.jpg')
).execute()
clear_output()
os.remove(client_secrect_key)
os.remove(pickle_file)
os.remove("image.jpg")
print("upload successful!")

upload successful!


In [2]:
# @title ##**Download youtube video or audio** { display-mode: "form" }
download_format="audio"#@param["video","audio"]
enter_youtube_link = "https://youtu.be/OkXttCJFL5g" #@param {type:"string"}
if download_format=="video":
  youtube_id=enter_youtube_link.split("/")
  dummy="youtube-dl --format mp4 https://www.youtube.com/watch?v="+youtube_id[-1]
  try:
    os.system(dummy)
    print("Youtube video download succesful")
  except:
    print("Can't download Youtube video!")
elif download_format=="audio":
  youtube_id=enter_youtube_link.split("/")
  new_link="https://www.youtube.com/watch?v="+youtube_id[-1]                                                            
  dummy='youtube-dl -f "bestaudio/best" -ciw -o "%(title)s.%(ext)s" -v --extract-audio --audio-quality 0 --audio-format mp3  '
  dummy=dummy+new_link
  with open("download_audio.sh","w") as f:
    f.write(dummy)
  !chmod 777 download_audio.sh  
  try:
    subprocess.call("./download_audio.sh", shell=True)
    print("Youtube audio download succesful")
  except:
    print("Can't download Youtube audio!")
  os.remove("download_audio.sh")



Youtube audio download succesful


In [3]:
# @title ##**Download the file on local device** { display-mode: "form" }

type_file_path="/content/download_from_colab/Google Maps design - Exploring color.mp3"#@param {type:"string"}
files.download(type_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ada